In [ ]:
# Downloads audio data for augmentation
# Borrowed from openWakeWord's automatic_model_training.ipynb, accessed March 4, 2024

import datasets
import scipy
import os

import numpy as np

from pathlib import Path
from tqdm import tqdm

## Download MIR RIR data

output_dir = "./mit_rirs"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    rir_dataset = datasets.load_dataset("davidscripka/MIT_environmental_impulse_responses", split="train", streaming=True)
    # Save clips to 16-bit PCM wav files
    for row in tqdm(rir_dataset):
        name = row['audio']['path'].split('/')[-1]
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

## Download noise and background audio

# Audioset Dataset (https://research.google.com/audioset/dataset/index.html)
# Download one part of the audioset .tar files, extract, and convert to 16khz
# For full-scale training, it's recommended to download the entire dataset from
# https://huggingface.co/datasets/agkphysics/AudioSet, and
# even potentially combine it with other background noise datasets (e.g., FSD50k, Freesound, etc.)

if not os.path.exists("audioset"):
    os.mkdir("audioset")

    fname = "bal_train09.tar"
    out_dir = f"audioset/{fname}"
    link = "https://huggingface.co/datasets/agkphysics/AudioSet/resolve/main/data/" + fname
    !wget -O {out_dir} {link}
    !cd audioset && tar -xvf bal_train09.tar

    output_dir = "./audioset_16k"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    # Save clips to 16-bit PCM wav files
    audioset_dataset = datasets.Dataset.from_dict({"audio": [str(i) for i in Path("audioset/audio").glob("**/*.flac")]})
    audioset_dataset = audioset_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000))
    for row in tqdm(audioset_dataset):
        name = row['audio']['path'].split('/')[-1].replace(".flac", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))

# Free Music Archive dataset
# https://github.com/mdeff/fma

output_dir = "./fma"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    fma_dataset = datasets.load_dataset("rudraml/fma", name="small", split="train", streaming=True)
    fma_dataset = iter(fma_dataset.cast_column("audio", datasets.Audio(sampling_rate=16000)))

    # Save clips to 16-bit PCM wav files
    n_hours = 1  # use only 1 hour of clips for this example notebook, recommend increasing for full-scale training
    for i in tqdm(range(n_hours*3600//30)):  # this works because the FMA dataset is all 30 second clips
        row = next(fma_dataset)
        name = row['audio']['path'].split('/')[-1].replace(".mp3", ".wav")
        scipy.io.wavfile.write(os.path.join(output_dir, name), 16000, (row['audio']['array']*32767).astype(np.int16))
        i += 1
        if i == n_hours*3600//30:
            break
        
        

In [ ]:
# Specify parameters for augmentation

audio_config = {}

audio_config['features_output_dir'] = 'augmented_features_mmap'

audio_config['input_path'] = 'generated_samples'
audio_config['input_glob'] = '**/*.wav'
audio_config['impulse_paths'] = ['mit_rirs']
audio_config['background_paths'] = ['fma', 'audioset_16k']
audio_config['min_clip_duration_s'] = None
audio_config['max_clip_duration_s'] = 1.39
audio_config['max_start_time_from_right_s'] = 1.49
audio_config['augmented_duration_s'] = 3.0

from microwakeword.feature_generation import ClipsHandler

In [ ]:
# Load audio clips and prepare them for augmentation

clips_handler = ClipsHandler(
                            input_path=audio_config['input_path'],
                            input_glob=audio_config['input_glob'],
                            impulse_paths=audio_config['impulse_paths'], 
                            background_paths=audio_config['background_paths'], 
                            augmentation_probabilities = {
                                "SevenBandParametricEQ": 0.25,
                                "TanhDistortion": 0.25,
                                "PitchShift": 0.25,
                                "BandStopFilter": 0.25,
                                "AddBackgroundNoise": 0.75,
                                "Gain": 1.0,
                                "RIR": 0.5,
                            },
                            augmented_duration_s = audio_config['augmented_duration_s'],
                            max_start_time_from_right_s = audio_config['max_start_time_from_right_s'],
                            max_clip_duration_s = audio_config['max_clip_duration_s'],   
                            min_clip_duration_s = audio_config['min_clip_duration_s'],     
)

In [ ]:
# Test by playing a randomly augmented clip

import IPython

clips_handler.save_random_augmented_clip("augmented_clip.wav")

IPython.display.display(IPython.display.Audio("augmented_clip.wav"))

In [ ]:
# Save all the clip's augmented features in a Ragged Mmap

clips_handler.save_augmented_features(audio_config['features_output_dir'])